# Creating FABnet IPv4 Network

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links. 

This notebook describes how to use the FABnetv4 service which is FABRIC's private IPv4 internet.   


## Import the FABlib Library


In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/work/fabric_config/tokens.json
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Bastion Username,pruth_0031379841
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add a separate `l3network` for each site and pass the list of interfaces on that site that you want to connect to FABnetv4. All interfaces passed to `l3network` must be on the same site and each network will be placed on that site.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on different sites you can specify the name of the sites in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a set of random site names that guarantee that the sites are different. 


NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [2]:
slice_name = 'MySliceXXXXX2'
[site1,site2] = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

Sites: MICH, CLEM


In [3]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Networks
net1 = slice.add_l3network(name=network1_name, type='IPv4')
net2 = slice.add_l3network(name=network2_name, type='IPv4')

# Node1
node1 = slice.add_node(name=node1_name, site=site1).enable_docker()
node1.set_rocky_repo(IPv4Address('10.133.130.2'))
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)
node1.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())

# Node2
node2 = slice.add_node(name=node2_name, site=site2).enable_docker()
node2.set_rocky_repo(IPv4Address('10.133.130.2'))
iface2  = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net2.add_interface(iface2)
node2.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net2.get_gateway())

#Submit Slice Request
slice.submit();


Retry: 12, Time: 443 sec


ID,4b9226db-6e59-4580-b5ff-1679532b4479
Name,MySliceXXXXX2
Lease Expiration (UTC),2023-02-27 18:04:17 +0000
Lease Start (UTC),2023-02-26 18:04:18 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
cbe29724-1fd6-42d3-bc19-257dc21987b5,Node1,2,8,10,default_rocky_8,qcow2,mich-w1.fabric-testbed.net,MICH,rocky,2607:f018:110:11:f816:3eff:feac:d9e3,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2607:f018:110:11:f816:3eff:feac:d9e3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ca02d0bd-b368-4000-aa9a-ea6d451a69c5,Node2,2,8,10,default_rocky_8,qcow2,clem-w2.fabric-testbed.net,CLEM,rocky,2620:103:a006:12:f816:3eff:fe04:37fb,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:103:a006:12:f816:3eff:fe04:37fb,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
e189d46a-2004-4bc6-b78d-92ad74610602,net1,L3,FABNetv4,MICH,10.131.1.0/24,10.131.1.1,Active,
94a9b6eb-d1ea-489d-82ac-9fc14dc6f9d4,net2,L3,FABNetv4,CLEM,10.136.1.0/24,10.136.1.1,Active,


Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address
Node1-nic1-p1,Node1,net1,100,auto,,02:B3:F5:DB:97:7C,eth1,eth1,10.131.1.2
Node2-nic1-p1,Node2,net2,100,auto,,02:7B:2E:86:34:1B,eth1,eth1,10.136.1.2



Time to print interfaces 450 seconds


## Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
for node in slice.get_nodes():
    print(node.get_name())
    
    directory = "/etc/yum.repos.d/"
    
    string_to_comment = "mirrorlist"
    cmd = f"sudo bash -c 'for file in {directory}*; do sed -i \"/^{string_to_comment}/s/^/# /g\" \"$file\"; done'"
    node.execute(cmd)

    string_to_uncomment = "#baseurl"
    cmd = f"sudo bash -c 'for file in {directory}*; do sed -i \"/^{string_to_uncomment}/s/^#//g\" \"$file\"; done'"
    node.execute(cmd)
    
    old_string = "dl.rockylinux.org"
    new_string = "10.133.130.2" # current fabnetv4 ip of my rocky repo mirror
    cmd = f"sudo bash -c 'for file in \"{directory}\"*; do sed -i \"s/{old_string}/{new_string}/g\" \"$file\"; done'"
    node.execute(cmd)

    #node.execute(f"sudo sh -c \"echo 10.133.130.2 dl.rockylinux.org >> /etc/hosts \" ; ")
    #echo '10.133.130.2 dl.rockylinux.org >> /etc/hosts' 
    
    
    
          
        

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    



## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()